In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [10]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from typing import Dict, List
import os
import matplotlib.pyplot as plt

def read_all_data(dir_path:str='processed_training_data') -> Dict[str, pd.DataFrame]:
    imu = pd.DataFrame()
    imu_t = pd.DataFrame()
    ann = pd.DataFrame()
    ann_t = pd.DataFrame()

    for f in os.listdir(dir_path):
        x = pd.read_csv(f'{dir_path}/{f}',header=None)
        if f[-7:] == '__x.csv':
            imu = pd.concat([imu, x], axis=1)
        
        elif f[-12:] == '__x_time.csv':
            imu_t = pd.concat([imu_t, x], axis=0)
            
        elif f[-7:] == '__y.csv':
            ann = pd.concat([ann, x], axis=0)
            
        elif f[-12:] == '__y_time.csv':
            ann_t = pd.concat([ann_t, x], axis=0)

    

    return {
        'imu': imu,
        'imu_t': imu_t.reset_index(drop=True),
        'ann': ann.reset_index(drop=True),
        'ann_t': ann_t.reset_index(drop=True)
    }


df_dict = read_all_data('TrainingData')





In [40]:
dir_path = 'TrainingData'
imu = pd.DataFrame()
imu_t = pd.DataFrame()
target = pd.DataFrame()
target_t = pd.DataFrame()
for f in sorted(os.listdir(dir_path)):
        if f[-7:] == '__x.csv':
            x = pd.read_csv(f'{dir_path}/{f}',
                        header=None,
                        names =['x_accel','y_accel','z_accel','x_gyro','y_gyro','z_gyro'],
                        dtype=np.float64)
            sub = f[-13:-10]
            sub = int(sub)
            x = x.assign(subject = sub)
            imu = pd.concat([imu, x], axis=0)

        elif f[-12:] == '__x_time.csv':
            x = pd.read_csv(f'{dir_path}/{f}',
                        header=None,
                        names =['x_time'],
                        dtype=np.float64)
            sub = f[-18:-15]
            sub = int(sub)
            x = x.assign(subject = sub)
            imu_t = pd.concat([imu_t, x], axis=0)

        elif f[-7:] == '__y.csv':
            x = pd.read_csv(f'{dir_path}/{f}',
                        header=None,
                        names =['target'],
                        dtype=np.int8)
            sub = f[-13:-10]
            sub = int(sub)
            
            x = x.assign(subject = sub)
            target = pd.concat([target, x], axis=0)

        elif f[-12:] == '__y_time.csv':
            x = pd.read_csv(f'{dir_path}/{f}',
                        header=None,
                        names =['y_time'],
                        dtype=np.float64)
            sub = f[-18:-15]
            sub = int(sub)
            print(sub)
            x = x.assign(subject = sub)
            target_t = pd.concat([target_t, x], axis=0)
            
print(imu_t.head())
print(imu.iloc[111000])
print(imu_t.iloc[111000])




1
1
1
1
1
1
1
1
2
2
2
2
2
3
3
3
4
4
5
5
5
6
6
6
7
7
7
7
8
   x_time  subject
0   0.000        1
1   0.025        1
2   0.050        1
3   0.075        1
4   0.100        1
x_accel   -3.729431
y_accel    9.023376
z_accel    4.498917
x_gyro    -0.705524
y_gyro    -1.156916
z_gyro    -0.509679
subject    1.000000
Name: 2938, dtype: float64
x_time     73.45
subject     1.00
Name: 2938, dtype: float64


In [13]:
df_dict['imu'].shape

(70171, 174)

In [ ]:
"""
df = df.apply(pd.to_numeric, errors='coerce')
df = df.dropna()
df = df.to_numpy()
df.shape
"""

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from typing import Dict, List
import os
import matplotlib.pyplot as plt

def read_all_data(dir_path:str='processed_training_data') -> Dict[str, pd.DataFrame]:
    imu = pd.DataFrame()
    imu_t = pd.DataFrame()
    ann = pd.DataFrame()
    ann_t = pd.DataFrame()

    for f in os.listdir(dir_path):
        x = pd.read_csv(f'{dir_path}/{f}')
        if f[-7:] == '__x.csv':
            imu = pd.concat([imu, x], axis=0)
        
        elif f[-12:] == '__x_time.csv':
            imu_t = pd.concat([imu_t, x], axis=0)
            
        elif f[-7:] == '__y.csv':
            ann = pd.concat([ann, x], axis=0)
            
        elif f[-12:] == '__y_time.csv':
            ann_t = pd.concat([ann_t, x], axis=0)

    df = pd.concat([imu, ann])
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.dropna()

    return df.reset_index(drop=True)

df = read_all_data('data')
df = df.to_numpy()


'''
ann = data_dict['ann'].to_numpy().flatten()
del data_dict # Remove to free memory

imu = np.delete(imu,123375,0)
ann = np.delete(ann,123375)

imu = np.asarray(imu).astype('float32')
ann = np.asarray(ann).astype('float32')
"""
"""
split = 0.8

split_idx = int(len(imu)*split)
X_train = imu[:split_idx]
X_val = imu[split_idx:]
y_train = ann[:split_idx]
y_val = ann[split_idx:]
'''

In [ ]:
column_names = ['user_id', 'activity', 'timestamp', 'x_axis', 'y_axis', 'z_axis']

df = pd.read_csv('WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt', header=None, names=column_names)
df.z_axis.replace(regex=True, inplace=True, to_replace=r';', value=r'')
df['z_axis'] = df.z_axis.astype(np.float64)
df.dropna(axis=0, how='any', inplace=True)

In [ ]:
for i in range(0,y_train.shape[0]):
    try:
      d = float(y_train[i])
    except ValueError:
        print("error on line: ",i)

In [ ]:
for i in range(0,X_train.shape[0]):
  for j in range(0,X_train.shape[1]):
      try:
        d = float(X_train[i][j])
      except ValueError:
          print("error on line: ",i)

In [ ]:
for i in range(0,X_val.shape[0]):
  for j in range(0,X_val.shape[1]):
      try:
        d = float(X_val[i][j])
      except ValueError:
          print("error on line: ",i)

In [ ]:
for i in range(0,y_val.shape[0]):
    try:
      d = float(y_val[i])
    except ValueError:
        print("error on line: ",i)

In [ ]:
from sklearn.preprocessing import RobustScaler

# scale_columns = ['x_axis', 'y_axis', 'z_axis']

scaler = RobustScaler()

scaler = scaler.fit(X_train)

df_train = scaler.transform(X_train)

# df_test.loc[:, scale_columns] = scaler.transform(df_test[scale_columns].to_numpy())

In [ ]:
from scipy import stats

def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X[i:(i + time_steps)]
        labels = y[i: i + time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 400
STEP = 80

X_train, y_train = create_dataset(
    imu, 
    ann, 
    TIME_STEPS, 
    STEP
)
"""
X_test, y_test = create_dataset(
    df_test[['x_axis', 'y_axis', 'z_axis']], 
    df_test.activity, 
    TIME_STEPS, 
    STEP
)
"""

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
#y_test = enc.transform(y_test)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128, 
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split = 0.2,
    shuffle=False
)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_cm(y_true, y_pred, class_names):
  cm = confusion_matrix(y_true, y_pred)
  fig, ax = plt.subplots(figsize=(18, 16)) 
  ax = sns.heatmap(
      cm, 
      annot=True, 
      fmt="d", 
      cmap=sns.diverging_palette(220, 20, n=7),
      ax=ax
  )

  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  ax.set_xticklabels(class_names)
  ax.set_yticklabels(class_names)
  b, t = plt.ylim() # discover the values for bottom and top
  b += 0.5 # Add 0.5 to the bottom
  t -= 0.5 # Subtract 0.5 from the top
  plt.ylim(b, t) # update the ylim(bottom, top) values
  plt.show() # ta-da!

In [ ]:
plot_cm(
  enc.inverse_transform(y_test),
  enc.inverse_transform(y_pred),
  enc.categories_[0]
)